In [14]:
import pandas as pd
import numpy as np
import os

# SpeechCLIP
base_path = "/data/user_data/sbharad2/SpeechCLIP/data"
embedding_read_path_pattern = (
    "{base_path}/Flickr8k.{csv_name}.token.txt.audio_embeddings/"
)

# FLICKR
csv_path_pattern = "{base_path}/flickr/{csv_name}.csv"

train_df = pd.read_csv(
    csv_path_pattern.format(base_path=base_path, csv_name="flickr_train_sampled")
)
dev_df = pd.read_csv(
    csv_path_pattern.format(base_path=base_path, csv_name="flickr_dev_sampled")
)

# change target type as list
train_df["target"] = train_df["target"].apply(eval)
dev_df["target"] = dev_df["target"].apply(eval)

print("Train size:", train_df.shape, "Test size:", dev_df.shape)

Train size: (2000, 6) Test size: (1000, 6)


In [3]:
from sklearn.linear_model import LinearRegression


target_vocabulary = {}


def _load_data(df, embedding_read_path, target_vocabulary=None):
    all_embeddings = []
    all_targets = []
    for i, r in df.iterrows():
        try:
            example_id = r["example_id"]
            embedding = np.load(os.path.join(embedding_read_path, example_id) + ".npy")
            all_embeddings.append(embedding)
            for tgt in r["target"]:
                if tgt not in target_vocabulary:
                    target_vocabulary[tgt] = len(target_vocabulary)
        except FileNotFoundError:
            print(f"File not found for {example_id}.npy")

    for i, r in df.iterrows():
        example_id = r["example_id"]
        if not os.path.exists(os.path.join(embedding_read_path, example_id) + ".npy"):
            continue
        target = np.zeros(len(target_vocabulary))
        for tgt in r["target"]:
            target[target_vocabulary[tgt]] = 1
        all_targets.append(target)

    all_embeddings = np.array(all_embeddings)
    all_targets = np.array(all_targets)
    print(f"Loaded {all_embeddings.shape} embeddings and {all_targets.shape} targets.")
    return all_embeddings, all_targets, target_vocabulary


X_train, y_train, target_vocabulary = _load_data(
    train_df,
    embedding_read_path_pattern.format(
        base_path=base_path, csv_name="flickr_train_sampled"
    ),
    target_vocabulary,
)
print("Training shapes")
print(X_train.shape, y_train.shape)
print(len(target_vocabulary), "Target vocabulary size.")

(
    X_dev,
    y_dev,
    target_vocabulary,
) = _load_data(
    dev_df,
    embedding_read_path_pattern.format(
        base_path=base_path, csv_name="flickr_dev_sampled"
    ),
    target_vocabulary,
)
print("Dev shapes")
print(X_dev.shape, y_dev.shape)
print(len(target_vocabulary), "Target vocabulary size.")

# Remove columns from y_dev if they are not present in y_train
print("Original dev and train target shapes", y_dev.shape, y_train.shape)
if y_dev.shape[1] > y_train.shape[1]:
    print("Removing columns from dev target")
    y_dev = y_dev[:, : y_train.shape[1]]
print("New dev and train target shapes", y_dev.shape, y_train.shape)

File not found for 1155138244_859fd6e079.jpg#0.npy
File not found for 1468103286_96a6e07029.jpg#0.npy
File not found for 1479857177_9d4a6f38fd.jpg#0.npy
File not found for 1643915227_9f48068772.jpg#0.npy
File not found for 1797554350_20998753c0.jpg#1.npy
File not found for 1808504612_3508f3c9bb.jpg#0.npy
File not found for 199463720_329a802206.jpg#0.npy
File not found for 2058091220_2087270068.jpg#0.npy
File not found for 2087317114_cf06df5aa5.jpg#1.npy
File not found for 2136455112_202c093ba4.jpg#0.npy
File not found for 2221818690_9003756d33.jpg#1.npy
File not found for 2258277193_586949ec62.jpg.1#2.npy
File not found for 2319197581_94f807b204.jpg#0.npy
File not found for 236095031_5cb17dc54a.jpg#0.npy
File not found for 2394824046_51cec8e5e7.jpg#1.npy
File not found for 240696675_7d05193aa0.jpg#0.npy
File not found for 2410153942_ba4a136358.jpg#0.npy
File not found for 2428275562_4bde2bc5ea.jpg#1.npy
File not found for 2553619107_d382a820f9.jpg#1.npy
File not found for 2557972410_69

In [4]:
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score

clf = MultiOutputClassifier(LogisticRegression()).fit(X_train, y_train)
y_pred = clf.predict(X_dev)
# Compute multi label metrics
print("Exact Match:", accuracy_score(y_dev, y_pred) * 100)

print("Micro averaged metrics")
print("F1 Score:", f1_score(y_dev, y_pred, average="micro") * 100)
print("Precision:", precision_score(y_dev, y_pred, average="micro") * 100)
print("Recall:", recall_score(y_dev, y_pred, average="micro") * 100)

Exact Match: 15.6
Micro averaged metrics
F1 Score: 44.66257668711657
Precision: 88.3495145631068
Recall: 29.88505747126437


# FAST-VGS

In [15]:
import pandas as pd
import numpy as np
import os

# SpeechCLIP
base_path = "/data/user_data/sbharad2/SpeechCLIP/data"
embedding_read_path_pattern = (
    "{base_path}/Flickr8k.{csv_name}.token.txt.audio_embeddings/"
)

# FLICKR
csv_path_pattern = "{base_path}/flickr/{csv_name}.csv"

train_df = pd.read_csv(
    csv_path_pattern.format(base_path=base_path, csv_name="flickr_train_sampled")
)
dev_df = pd.read_csv(
    csv_path_pattern.format(base_path=base_path, csv_name="flickr_dev_sampled")
)

# change target type as list
train_df["target"] = train_df["target"].apply(eval)
dev_df["target"] = dev_df["target"].apply(eval)

print("Train size:", train_df.shape, "Test size:", dev_df.shape)

Train size: (2000, 6) Test size: (1000, 6)


In [22]:
import numpy as np

f_train_embedding_path = "/data/user_data/sbharad2/SpeechCLIP/data/Fastvgs_embeddings/embeddings_flickr_train_matrix.npy"
f_train_txt_path = "/data/user_data/sbharad2/SpeechCLIP/data/Fastvgs_embeddings/embeddings_flickr_train.txt"

f_dev_embedding_path = "/data/user_data/sbharad2/SpeechCLIP/data/Fastvgs_embeddings/embeddings_flickr_dev_matrix_clean.npy"
f_dev_txt_path = "/data/user_data/sbharad2/SpeechCLIP/data/Fastvgs_embeddings/embeddings_flickr_dev_clean.txt"


def _transform_wav_name_to_example_id(wav_name):
    wav_name = wav_name.replace(".wav", "")
    caption_id = wav_name[-1]
    img_id = wav_name[:-2]
    return f"{img_id}.jpg#{caption_id}"


def _read_embeddings_as_dict(embedding_path, txt_path):
    f_emb = np.load(embedding_path)
    with open(txt_path, "r") as f:
        names = [line.strip() for line in f.readlines()]
        names = [_transform_wav_name_to_example_id(name) for name in names]
    emb_dict = {name: emb for name, emb in zip(names, f_emb)}
    # normalize
    emb_dict = {k: v / np.linalg.norm(v) for k, v in emb_dict.items()}
    return emb_dict

In [23]:
train_embeddings = _read_embeddings_as_dict(f_train_embedding_path, f_train_txt_path)
dev_embeddings = _read_embeddings_as_dict(f_dev_embedding_path, f_dev_txt_path)

In [25]:
def _load_data_fastvgs(df, embeddings_dict, target_vocabulary=None):
    all_embeddings = []
    all_targets = []
    for i, r in df.iterrows():
        try:
            example_id = r["example_id"]
            print(example_id)
            if example_id not in embeddings_dict:
                continue
            embedding = embeddings_dict[example_id]
            all_embeddings.append(embedding)
            for tgt in r["target"]:
                if tgt not in target_vocabulary:
                    target_vocabulary[tgt] = len(target_vocabulary)
        except FileNotFoundError:
            print(f"File not found for {example_id}.npy")

    for i, r in df.iterrows():
        example_id = r["example_id"]
        if example_id not in embeddings_dict:
            continue
        target = np.zeros(len(target_vocabulary))
        for tgt in r["target"]:
            target[target_vocabulary[tgt]] = 1
        all_targets.append(target)

    all_embeddings = np.array(all_embeddings)
    all_targets = np.array(all_targets)
    print(f"Loaded {all_embeddings.shape} embeddings and {all_targets.shape} targets.")
    return all_embeddings, all_targets, target_vocabulary


target_vocabulary = {}
X_train, y_train, target_vocabulary = _load_data_fastvgs(
    train_df,
    train_embeddings,
    target_vocabulary,
)
print("Training shapes")
print(X_train.shape, y_train.shape)
print(len(target_vocabulary), "Target vocabulary size.")

(
    X_dev,
    y_dev,
    target_vocabulary,
) = _load_data_fastvgs(
    dev_df,
    dev_embeddings,
    target_vocabulary,
)
print("Dev shapes")
print(X_dev.shape, y_dev.shape)
print(len(target_vocabulary), "Target vocabulary size.")

# Remove columns from y_dev if they are not present in y_train
print("Original dev and train target shapes", y_dev.shape, y_train.shape)
if y_dev.shape[1] > y_train.shape[1]:
    print("Removing columns from dev target")
    y_dev = y_dev[:, : y_train.shape[1]]
print("New dev and train target shapes", y_dev.shape, y_train.shape)

1000268201_693b08cb0e.jpg#0
1001773457_577c3a7d70.jpg#0
1002674143_1b742ab4b8.jpg#0
1003163366_44323f5815.jpg#0
1007129816_e794419615.jpg#0
1007320043_627395c3d8.jpg#0
1009434119_febe49276a.jpg#0
1012212859_01547e3f17.jpg#0
1015118661_980735411b.jpg#0
1015584366_dfcec3c85a.jpg#0
101654506_8eb26cfb60.jpg#0
101669240_b2d3e7f17b.jpg#0
1016887272_03199f49c4.jpg#0
1019077836_6fc9b15408.jpg#0
1019604187_d087bf9a5f.jpg#0
1020651753_06077ec457.jpg#0
1022454428_b6b660a67b.jpg#0
1022975728_75515238d8.jpg#0
102351840_323e3de834.jpg#0
1024138940_f1fefbdce1.jpg#0
102455176_5f8ead62d5.jpg#0
1026685415_0431cbf574.jpg#0
1028205764_7e8df9a2ea.jpg#0
1030985833_b0902ea560.jpg#0
103195344_5d2dc613a3.jpg#0
103205630_682ca7285b.jpg#1
1032460886_4a598ed535.jpg#0
104136873_5b5d41be75.jpg#0
1042020065_fb3d3ba5ba.jpg#0
1042590306_95dea0916c.jpg#0
1048710776_bb5b0a5c7c.jpg#0
1052358063_eae6744153.jpg#0
105342180_4d4a40b47f.jpg#0
1053804096_ad278b25f1.jpg#0
1055623002_8195a43714.jpg#0
1055753357_4fa3d8d693.jpg#1


In [26]:
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score

clf = MultiOutputClassifier(LogisticRegression()).fit(X_train, y_train)
y_pred = clf.predict(X_dev)
# Compute multi label metrics
print("Exact Match:", accuracy_score(y_dev, y_pred) * 100)

print("Micro averaged metrics")
print("F1 Score:", f1_score(y_dev, y_pred, average="micro") * 100)
print("Precision:", precision_score(y_dev, y_pred, average="micro") * 100)
print("Recall:", recall_score(y_dev, y_pred, average="micro") * 100)

Exact Match: 10.2
Micro averaged metrics
F1 Score: 36.54694391934467
Precision: 78.59078590785907
Recall: 23.809523809523807
